# Agilepy science API tutorial

You DON'T need all the AGILE data archive to run this notebook!

## Importing the library

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
from os.path import expandvars
from pathlib import Path
from shutil import rmtree

In [ ]:
%matplotlib notebook

## Output directory 

In [ ]:
outputFolder = Path(expandvars("$HOME/agilepy_science_api_tutorial/output"))

if Path(outputFolder).exists():
    print("Deleting",outputFolder)
    rmtree(outputFolder)

## Configuration file

In [ ]:
config = """
input:
  evtfile: $AGILE/agilepy-test-data/evt_index/agile_proc3_fm3.119_asdc2_EVT.index
  logfile: $AGILE/agilepy-test-data/log_index/agile_proc3_data_asdc2_LOG.log.index
  # evtfile: /AGILE_PROC3/FM3.119_ASDC2/INDEX/EVT.index
  # logfile: /AGILE_PROC3/DATA_ASDC2/INDEX/LOG.log.index

output:
  outdir: %s
  filenameprefix: science_api_tutorial
  logfilenameprefix: science_api_tutorial_log
  verboselvl: 1

selection:
  tmin: 456361778
  tmax: 456537945
  timetype: TT
  glon: 80.0 
  glat: 1.0
 
maps:
  energybins:
    - 100, 300
    - 300, 1000
  fovbinnumber: 2

model:
  galcoeff: null
  isocoeff: null
  
"""%(outputFolder)

with open("./agilepy_conf.yaml", "w") as cf:
    cf.write(config)

## Initialization of the class

In [ ]:
ag = AGAnalysis("./agilepy_conf.yaml")

## API: configuration file 

In [ ]:
ag.printOptions()

In [ ]:
ag.printOptions("maps")

In [ ]:
ag.getOption("energybins")

In [ ]:
ag.setOptions(energybins=[[100,300], [300, 1000]], mapsize=50, binsize=0.4)

In [ ]:
ag.printOptions("maps")

## API: sources

In [ ]:
cat2agl = ag.getSupportedCatalogs().pop()
cat2agl

In [ ]:
!head -n 5 "$AGILE/catalogs/2AGL.multi"

In [ ]:
sources = ag.loadSources(cat2agl, rangeDist = (0, 5))

In [ ]:
type(sources[0])

In [ ]:
for s in sources:
    print(s)

### Selecting the sources
The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

In [ ]:
selectedSources = ag.selectSources('flux > 0')

In [ ]:
len(selectedSources)

In [ ]:
selectedSources = ag.selectSources('name == "2AGLJ2021+4029"')

In [ ]:
len(selectedSources)

In [ ]:
selectedSources = ag.selectSources("flux > 0 AND dist <= 2", quiet=True)

In [ ]:
len(selectedSources)

### Free/Fix source parameters
You can fix or free the following parametes:

* flux
* index
* index1
* index2
* cutoffEnergy
* pivotEnergy
* curvature
* index2

In [ ]:
_sources = ag.freeSources('name == "2AGLJ2021+4029"', "flux", True)

In [ ]:
for s in _sources:
    print(s)

In [ ]:
_sources = ag.freeSources('name == "2AGLJ2021+4029"', "index", True)

In [ ]:
for s in _sources:
    print(s)

In [ ]:
_sources = ag.freeSources("flux > 0 AND dist <= 2", "flux", True)

In [ ]:
len(_sources)

In [ ]:
for s in _sources:
    print(s)

Resetting the changes:

In [ ]:
_sources = ag.freeSources('name == "2AGLJ2021+4029"', "flux", False)

In [ ]:
ag.freeSources('name == "2AGLJ2021+4029"', "index", False)

In [ ]:
ag.freeSources("flux > 0 AND dist <= 2", "flux", False)

### Deleting sources

In [ ]:
deleted = ag.deleteSources('name == "2AGLJ2029+4403"').pop()
print(deleted)

In [ ]:
len(ag.getSources())

### Adding a source

Passing a dictionary:

In [ ]:
newSourceDict = {
    "glon" : 81.9171,
    "glat": 3.0854,
    "spectrumType" : "PowerLaw",
    "flux": 9.97261e-08,
    "index": 1.92501
}

if ag.addSource("2AGLJ2029+4403", newSourceDict):
    print("Source loaded")

In [ ]:
len(ag.getSources())

Passing a Source object:

In [ ]:
type(deleted)

In [ ]:
if ag.addSource("2AGLJ2029+4403", deleted):
    print("Source loaded")
else:
    print("Source is already present in the SourcesLibrary")

In [ ]:
deletedSources = ag.deleteSources('name == "2AGLJ2029+4403"')

In [ ]:
len(ag.getSources())

## API: science

### Generating maps

In [ ]:
ag.printOptions("maps")

In [ ]:
maplistfile = ag.generateMaps()

### Displaying maps

#### Single figure - multiple subplots

In [ ]:
ag.displayCtsSkyMaps(smooth=True, sigma=2, regFilePath="$AGILE/catalogs/2AGL_2.reg")

#### Multiple figures - one subplot per figure

In [ ]:
ag.displayExpSkyMaps(singleMode = False)

In [ ]:
ag.displayGasSkyMaps()

### Saving maps

In [ ]:
ag.displayCtsSkyMaps(smooth=True, sigma=3)

### Estimation of background coefficients

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

In [ ]:
# This method will require data coming from 14 days before tmin. Hence, you can't execute this method if you are using: 

# evtfile: $AGILE/agilepy-test-data/evt_index/agile_proc3_fm3.119_asdc2_EVT.index
# logfile: $AGILE/agilepy-test-data/log_index/agile_proc3_data_asdc2_LOG.log.index

# ag.calcBkg("2AGLJ2021+4029")

ag.setOptions(galcoeff=[0.469176, 0.699403, 0.199108, 0.533379], isocoeff=[4.27314, 0.976644, 8.12386, 1.24627])

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

### Maximum likelyhood analysis

In [ ]:
ag.freeSources('name == "2AGLJ2021+4029"', "flux", True)

In [ ]:
sourceFiles = ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")
len(sources)

In [ ]:
source = ag.freeSources('name == "2AGLJ2021+4029"', "pos", True)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")